In [97]:
import pandas as pd

In [98]:
pd.options.display.max_colwidth = 150

In [99]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\fo18OCT2019bhav.csv")

Preview of file imported as Dataframe

In [100]:
df.head(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,Unnamed: 15
0,FUTIDX,BANKNIFTY,31-Oct-2019,0.0,XX,29029.6,29299.95,28903.05,29152.35,29152.35,192396,1120156.53,1295200,-129540,18-OCT-2019,NaN
1,FUTIDX,BANKNIFTY,28-Nov-2019,0.0,XX,29082.7,29405.00,29022.75,29270.10,29270.10,5125,29962.97,154240,9920,18-OCT-2019,NaN
2,FUTIDX,BANKNIFTY,26-Dec-2019,0.0,XX,29150.0,29505.50,29140.00,29371.70,29371.70,360,2112.77,12080,360,18-OCT-2019,NaN


Dropping unnamed column

In [101]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

Filtering stocks where volume traded is zero, and dropping un-required columns from data frame

In [103]:
df = df[df['CONTRACTS']>0]

In [104]:
df.drop(('OPEN HIGH LOW SETTLE_PR CONTRACTS VAL_INLAKH OPEN_INT CHG_IN_OI TIMESTAMP'.split()), axis = 1, inplace=True)

In [105]:
df.head(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE
0,FUTIDX,BANKNIFTY,31-Oct-2019,0.0,XX,29152.35
1,FUTIDX,BANKNIFTY,28-Nov-2019,0.0,XX,29270.10
2,FUTIDX,BANKNIFTY,26-Dec-2019,0.0,XX,29371.70


Creating new tickers, so as to identify all strikes (of all expiries) uniquely

In [106]:
df['CE'] = "CE"
df['PE'] = "PE"
df['Ticker'] = df['INSTRUMENT'] + " " +  df["SYMBOL"] + " " + df["EXPIRY_DT"] + " " + df['STRIKE_PR'].map(str) + " " + df['OPTION_TYP']
df['CALL_TICKER'] = df['INSTRUMENT'] + " " + df["SYMBOL"] + " " + df["EXPIRY_DT"] + " " + (df[df['STRIKE_PR']> 0]['STRIKE_PR']).map(str) + " " + (df[df['OPTION_TYP']!= 'XX']["CE"])
df['PUT_TICKER'] = df['INSTRUMENT'] + " " + df["SYMBOL"] + " " + df["EXPIRY_DT"] + " " + (df[df['STRIKE_PR']> 0]['STRIKE_PR']).map(str) + " " + (df[df['OPTION_TYP']!= 'XX']["PE"])

In [107]:
df.tail(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE,CE,PE,Ticker,CALL_TICKER,PUT_TICKER
36276,OPTSTK,ZEEL,28-Nov-2019,200.0,PE,9.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,OPTSTK ZEEL 28-Nov-2019 200.0 CE,OPTSTK ZEEL 28-Nov-2019 200.0 PE
36280,OPTSTK,ZEEL,28-Nov-2019,240.0,PE,22.5,CE,PE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,OPTSTK ZEEL 28-Nov-2019 240.0 CE,OPTSTK ZEEL 28-Nov-2019 240.0 PE
36281,OPTSTK,ZEEL,28-Nov-2019,250.0,PE,30.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,OPTSTK ZEEL 28-Nov-2019 250.0 CE,OPTSTK ZEEL 28-Nov-2019 250.0 PE


In [108]:
instr_type = {"OPTSTK":"FUTSTK", "OPTIDX":"FUTIDX"}
df['FUT1_TICKER'] = df["INSTRUMENT"].map(instr_type) + " " + df['SYMBOL'] + " " + df["EXPIRY_DT"] + " " + "0.0 XX"

In [109]:
df.tail(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE,CE,PE,Ticker,CALL_TICKER,PUT_TICKER,FUT1_TICKER
36276,OPTSTK,ZEEL,28-Nov-2019,200.0,PE,9.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,OPTSTK ZEEL 28-Nov-2019 200.0 CE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX
36280,OPTSTK,ZEEL,28-Nov-2019,240.0,PE,22.5,CE,PE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,OPTSTK ZEEL 28-Nov-2019 240.0 CE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX
36281,OPTSTK,ZEEL,28-Nov-2019,250.0,PE,30.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,OPTSTK ZEEL 28-Nov-2019 250.0 CE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX


index - match of  call - put - fut prices in one row

In [110]:
mydict = dict(zip(df.Ticker, df.CLOSE))

In [111]:
df["Call_Price"] = df['CALL_TICKER'].map(mydict)
df["Put_Price"] = df['PUT_TICKER'].map(mydict)
df["Fut_Price"] = df['FUT1_TICKER'].map(mydict)

In [112]:
df.tail(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE,CE,PE,Ticker,CALL_TICKER,PUT_TICKER,FUT1_TICKER,Call_Price,Put_Price,Fut_Price
36276,OPTSTK,ZEEL,28-Nov-2019,200.0,PE,9.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,OPTSTK ZEEL 28-Nov-2019 200.0 CE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,9.9,251.75
36280,OPTSTK,ZEEL,28-Nov-2019,240.0,PE,22.5,CE,PE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,OPTSTK ZEEL 28-Nov-2019 240.0 CE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,22.5,251.75
36281,OPTSTK,ZEEL,28-Nov-2019,250.0,PE,30.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,OPTSTK ZEEL 28-Nov-2019 250.0 CE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,30.9,251.75


performing calculation for Put-Call - Parity

In [113]:
df['Put_Call_Parity'] = (df['Put_Price']*-1) + (df['Fut_Price']*-1) + df['Call_Price'] + df['STRIKE_PR']

In [114]:
df['Put_Call_Parity_%'] = df['Put_Call_Parity'] / df['Fut_Price']

In [115]:
df.tail(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE,CE,PE,Ticker,CALL_TICKER,PUT_TICKER,FUT1_TICKER,Call_Price,Put_Price,Fut_Price,Put_Call_Parity,Put_Call_Parity_%
36276,OPTSTK,ZEEL,28-Nov-2019,200.0,PE,9.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,OPTSTK ZEEL 28-Nov-2019 200.0 CE,OPTSTK ZEEL 28-Nov-2019 200.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,9.9,251.75,NaN,NaN
36280,OPTSTK,ZEEL,28-Nov-2019,240.0,PE,22.5,CE,PE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,OPTSTK ZEEL 28-Nov-2019 240.0 CE,OPTSTK ZEEL 28-Nov-2019 240.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,22.5,251.75,NaN,NaN
36281,OPTSTK,ZEEL,28-Nov-2019,250.0,PE,30.9,CE,PE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,OPTSTK ZEEL 28-Nov-2019 250.0 CE,OPTSTK ZEEL 28-Nov-2019 250.0 PE,FUTSTK ZEEL 28-Nov-2019 0.0 XX,NaN,30.9,251.75,NaN,NaN


sorting dataframe by Put-Call-Parity difference % in descending order

In [116]:
df = df.sort_values("Put_Call_Parity_%", ascending=False)

In [117]:
df['Remarks'] = "BUY" + " " + df["PUT_TICKER"] + " " + "BUY" + " " + df["FUT1_TICKER"] + " " + "SELL" + " " + df["CALL_TICKER"]

In [118]:
df.head(3)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,CLOSE,CE,PE,Ticker,CALL_TICKER,PUT_TICKER,FUT1_TICKER,Call_Price,Put_Price,Fut_Price,Put_Call_Parity,Put_Call_Parity_%,Remarks
24685,OPTSTK,MOTHERSUMI,28-Nov-2019,105.0,CE,16.75,CE,PE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE,FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX,16.75,5.0,108.05,8.70,0.080518,BUY OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE BUY FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX SELL OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE
24717,OPTSTK,MOTHERSUMI,28-Nov-2019,105.0,PE,5.00,CE,PE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE,OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE,FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX,16.75,5.0,108.05,8.70,0.080518,BUY OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE BUY FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX SELL OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE
10237,OPTSTK,BHEL,31-Oct-2019,40.0,PE,0.10,CE,PE,OPTSTK BHEL 31-Oct-2019 40.0 PE,OPTSTK BHEL 31-Oct-2019 40.0 CE,OPTSTK BHEL 31-Oct-2019 40.0 PE,FUTSTK BHEL 31-Oct-2019 0.0 XX,17.50,0.1,54.55,2.85,0.052246,BUY OPTSTK BHEL 31-Oct-2019 40.0 PE BUY FUTSTK BHEL 31-Oct-2019 0.0 XX SELL OPTSTK BHEL 31-Oct-2019 40.0 CE


creating a brand new data_frame from above dataframe, so that to reflect only required columns and stock with maximum p-c-parity % on top

In [119]:
df_final = df[['SYMBOL', 'EXPIRY_DT', 'Put_Call_Parity_%', 'Remarks']]

In [120]:
pd.options.display.max_colwidth = 150

displaying top 20 stocks in the dataframe - along with the position to be taken under 'Remarks' Column - 

In [121]:
df_final.head(20)

,SYMBOL,EXPIRY_DT,Put_Call_Parity_%,Remarks
24685,MOTHERSUMI,28-Nov-2019,0.080518,BUY OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE BUY FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX SELL OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE
24717,MOTHERSUMI,28-Nov-2019,0.080518,BUY OPTSTK MOTHERSUMI 28-Nov-2019 105.0 PE BUY FUTSTK MOTHERSUMI 28-Nov-2019 0.0 XX SELL OPTSTK MOTHERSUMI 28-Nov-2019 105.0 CE
10237,BHEL,31-Oct-2019,0.052246,BUY OPTSTK BHEL 31-Oct-2019 40.0 PE BUY FUTSTK BHEL 31-Oct-2019 0.0 XX SELL OPTSTK BHEL 31-Oct-2019 40.0 CE
10187,BHEL,31-Oct-2019,0.052246,BUY OPTSTK BHEL 31-Oct-2019 40.0 PE BUY FUTSTK BHEL 31-Oct-2019 0.0 XX SELL OPTSTK BHEL 31-Oct-2019 40.0 CE
10188,BHEL,31-Oct-2019,0.034830,BUY OPTSTK BHEL 31-Oct-2019 41.0 PE BUY FUTSTK BHEL 31-Oct-2019 0.0 XX SELL OPTSTK BHEL 31-Oct-2019 41.0 CE
10238,BHEL,31-Oct-2019,0.034830,BUY OPTSTK BHEL 31-Oct-2019 41.0 PE BUY FUTSTK BHEL 31-Oct-2019 0.0 XX SELL OPTSTK BHEL 31-Oct-2019 41.0 CE
18870,ICICIBANK,31-Oct-2019,0.029740,BUY OPTSTK ICICIBANK 31-Oct-2019 380.0 PE BUY FUTSTK ICICIBANK 31-Oct-2019 0.0 XX SELL OPTSTK ICICIBANK 31-Oct-2019 380.0 CE
18836,ICICIBANK,31-Oct-2019,0.029740,BUY OPTSTK ICICIBANK 31-Oct-2019 380.0 PE BUY FUTSTK ICICIBANK 31-Oct-2019 0.0 XX SELL OPTSTK ICICIBANK 31-Oct-2019 380.0 CE
18729,IBULHSGFIN,28-Nov-2019,0.027929,BUY OPTSTK IBULHSGFIN 28-Nov-2019 220.0 PE BUY FUTSTK IBULHSGFIN 28-Nov-2019 0.0 XX SELL OPTSTK IBULHSGFIN 28-Nov-2019 220.0 CE
18689,IBULHSGFIN,28-Nov-2019,0.027929,BUY OPTSTK IBULHSGFIN 28-Nov-2019 220.0 PE BUY FUTSTK IBULHSGFIN 28-Nov-2019 0.0 XX SELL OPTSTK IBULHSGFIN 28-Nov-2019 220.0 CE
